In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
# Initialize Inception model
import torchvision.models as models
import numpy as np

In [2]:
pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 40.9 MB/s eta 0:00:00


In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [39]:
# Data augmentation and normalization for training
# Just normalization for validation
# Data augmentation and normalization for training
train_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

val_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [31]:
import torchvision
# Use the transforms in your dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=train_transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers=4)

val_dataset = CIFAR10(root='./data', train=False, transform=val_transform, download=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [66]:
# Initialize ResNet50 model
import torchvision.models as models

# Load pre-trained VGG16 model
model = torchvision.models.vgg16(pretrained=True)
# Modify the last layer for 10 classes
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 10)

# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False


In [67]:
import torch
import pennylane as qml

dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def quantum_net(q_input_features, q_weights_flat):
    # Reshape weights
    q_weights = q_weights_flat.reshape((4, 3))  # Assuming 3 parameters per qubit

    # Quantum circuit
    for i in range(4):
        qml.Rot(q_input_features[i][0], q_input_features[i][1], q_input_features[i][2], wires=i)
        qml.RX(q_weights[i][0], wires=i)
        qml.RY(q_weights[i][1], wires=i)
        qml.RZ(q_weights[i][2], wires=i)

    return [qml.expval(qml.PauliZ(i)) for i in range(4)]




In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Inside the QuantumNetLayer class
class QuantumNetLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.q_weights_flat = nn.Parameter(torch.randn(4, 3).to(device))  # Define trainable quantum weights
        self.fc = nn.Linear(4, output_size).to(device)  # Adjust the input size to match the output of the quantum circuit

    def forward(self, x):
        q_features = quantum_net(torch.flatten(x, start_dim=1), self.q_weights_flat)
        q_features_tensor = torch.tensor(q_features, dtype=torch.float32).unsqueeze(0).to(device)
        x = self.fc(q_features_tensor)
        return x

# Assuming the output_size is known
quantum_layer = QuantumNetLayer(input_size=2048, output_size=10)  # Adjust the input size to match the output of the quantum circuit

# Replace the fully connected layer in the VGG16 model with the QuantumNetLayer
model.classifier = quantum_layer




In [69]:
# Set the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [70]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [71]:
# Loss function and optimizer
import torch.optim as optim
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


In [78]:
"""
# Training loop
num_epochs = 5
total_step = len(train_loader)
model = model.to(device)

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Ensure outputs and labels have the same batch size
        labels = labels[:outputs.shape[0]]  # Trim labels if needed to match batch size
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        print(f'Training: Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')
        # Adjust learning rate
    # scheduler.step()
"""

# Training loop
num_epochs = 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Ensure outputs and labels have the same batch size
        labels = labels[:outputs.shape[0]]  # Trim labels if needed to match batch size
        loss = criterion(outputs, labels)

        # L1 regularization
        l1_regularization = torch.tensor(0., requires_grad=True)
        for param in model.parameters():
            l1_regularization = l1_regularization + torch.norm(param, 1)

        loss = loss + 0.000000001 * l1_regularization  # Adjust the regularization strength (0.01 in this example)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        print(f'Training: Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')

Training: Epoch [1/5], Step [1/782], Loss: 2.4809088706970215
Training: Epoch [1/5], Step [2/782], Loss: 2.2347323894500732
Training: Epoch [1/5], Step [3/782], Loss: 2.7215447425842285
Training: Epoch [1/5], Step [4/782], Loss: 2.5797784328460693
Training: Epoch [1/5], Step [5/782], Loss: 2.187952995300293
Training: Epoch [1/5], Step [6/782], Loss: 2.700772285461426
Training: Epoch [1/5], Step [7/782], Loss: 2.1818456649780273
Training: Epoch [1/5], Step [8/782], Loss: 2.4635839462280273
Training: Epoch [1/5], Step [9/782], Loss: 3.037569999694824
Training: Epoch [1/5], Step [10/782], Loss: 2.6220781803131104
Training: Epoch [1/5], Step [11/782], Loss: 2.4616596698760986
Training: Epoch [1/5], Step [12/782], Loss: 1.8439446687698364
Training: Epoch [1/5], Step [13/782], Loss: 2.699443817138672
Training: Epoch [1/5], Step [14/782], Loss: 2.228158712387085
Training: Epoch [1/5], Step [15/782], Loss: 1.9641785621643066
Training: Epoch [1/5], Step [16/782], Loss: 3.048644781112671


KeyboardInterrupt: 

In [ ]:
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


NameError: name 'transform' is not defined

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on the test set: {100 * accuracy:.2f}%')


Accuracy on the test set: 9.27%
